# Holt Winters Time Series Analysis: NY-Births

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load the data

In [ ]:
birthData = pd.read_csv("../data/nybirths.txt", header=None)

In [ ]:
birthData.head()

In [ ]:
birthData.size

## Data preparation

In [ ]:
rng = pd.date_range('1960', periods=birthData.size, freq='MS')

In [ ]:
rng = pd.to_datetime(rng)

In [ ]:
df = pd.DataFrame({ 'Date': rng, 'Val':birthData[0] }) 

In [ ]:
df.head()

In [ ]:
df = df.set_index('Date')

In [ ]:
df.plot()

In [ ]:
# The moving average

In [ ]:
df['SMA'] = df['Val'].rolling(8).mean()

In [ ]:
print(df)

In [ ]:
df[['Val', 'SMA']].plot(label='Moving Average', 
                                  figsize=(16, 8))

# Time Series Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Get rid of the 'SMA' column as we don't need it
df = df.drop('SMA', axis=1)

In [ ]:
df

In [ ]:
# We multiply the values by 1000 and convert to integer because
# the decomposition function expects integers
df['Val'] = df['Val']*1000
df['Val'] = df['Val'].astype('int32')

In [ ]:
df

In [ ]:
#Perform the decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
series = df['Val']
result = seasonal_decompose(series, model='additive', period=12, extrapolate_trend=12)
print(result.trend)
print(result.seasonal)
print(result.resid)
print(result.observed)

In [ ]:
result.plot()
plt.show()

In [ ]:
#Create a column for the data after adjusting for seasonality
df['Adjusted']=df['Val']-result.seasonal

In [ ]:
df

In [ ]:
df.plot()

# Holt Winters Predictions

In [ ]:
# holt winters 
# single exponential smoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
# double and triple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
period  = 12
alpha = 1/(2*period)

In [ ]:
df

In [ ]:
# Simple exponential smoothing does not handle seasonality
df['HWES1'] = SimpleExpSmoothing(df['Val']).fit(smoothing_level=alpha,optimized=False,use_brute=True).fittedvalues 
df_tmp = df[['Val','HWES1']]
df_tmp.plot()

In [ ]:
# Break the data into a training set and a test set
train_births = df[:140]
test_births = df[140:]

In [ ]:
# Fit the model

fitted_model = SimpleExpSmoothing(train_births['Val']).fit()
test_predictions = fitted_model.forecast(30)
train_births['Val'].plot(legend=True,label='TRAIN')
test_births['Val'].plot(legend=True,label='TEST',figsize=(12,6))
test_predictions.plot(legend=True,label='PREDICTION')
plt.title('Train, Test and Predicted data points using Simple Exponential Smoothing')

In [ ]:
# Fit the model for triple exponential smoothing. This will handle seasonality

fitted_model = ExponentialSmoothing(train_births['Val'],trend='add',seasonal='add',seasonal_periods=12).fit()

test_predictions = fitted_model.forecast(30)

train_births['Val'].plot(legend=True,label='TRAIN')

test_births['Val'].plot(legend=True,label='TEST',figsize=(12,6))

test_predictions.plot(legend=True,label='PREDICTION')

plt.title('Train, Test and Predicted data points using Holt Winters Exponential Smoothing')